# course_3_project OMDB and TasteDive Mashup

This project will take you through the process of mashing up data from two different APIs to make movie recommendations. The TasteDive API lets you provide a movie (or bands, TV shows, etc.) as a query input, and returns a set of related items. The OMDB API lets you provide a movie title as a query input and get back data about the movie, including scores from various review sites (Rotten Tomatoes, IMDB, etc.).

You will put those two together. You will use TasteDive to get related movies for a whole list of titles. You’ll combine the resulting lists of related movies, and sort them according to their Rotten Tomatoes scores (which will require making API calls to the OMDB API.)

To avoid problems with rate limits and site accessibility, we have provided a cache file with results for all the queries you need to make to both OMDB and TasteDive. Just use requests_with_caching.get() rather than requests.get(). If you’re having trouble, you may not be formatting your queries properly, or you may not be asking for data that exists in our cache. We will try to provide as much information as we can to help guide you to form queries for which data exists in the cache.

Your first task will be to fetch data from TasteDive. The documentation for the API is at https://tastedive.com/read/api.

Define a function, called get_movies_from_tastedive. It should take one input parameter, a string that is the name of a movie or music artist. The function should return the 5 TasteDive results that are associated with that string; be sure to only get movies, not other kinds of media. It will be a python dictionary with just one key, ‘Similar’.

Try invoking your function with the input “Black Panther”.

HINT: Be sure to include only q, type, and limit as parameters in order to extract data from the cache. If any other parameters are included, then the function will not be able to recognize the data that you’re attempting to pull from the cache. Remember, you will not need an api key in order to complete the project, because all data will be found in the cache.

Define a function get_sorted_recommendations. It takes a list of movie titles as an input. It returns a sorted list of related movie titles as output, up to five related movies for each input movie title. The movies should be sorted in descending order by their Rotten Tomatoes rating, as returned by the get_movie_rating function. Break ties in reverse alphabetic order, so that ‘Yahşi Batı’ comes before ‘Eyyvah Eyvah’.

In [ ]:
import requests_with_caching as rq
def get_movies_from_tastedive(strng):
    params= {'q':strng,'type':'movies','limit':5}
    data = rq.get('https://tastedive.com/api/similar',params)
    response = data.json()
    return response
def extract_movie_titles(dic):
    movies=[]
    for items in dic['Similar']['Results']:
        movies.append(items['Name'])
    return movies
def get_related_titles(list_of_mov):
    lst =[]
    for movies in list_of_mov:
        movies_list=extract_movie_titles(get_movies_from_tastedive(movies))
        for item in movies_list:
            if item not in lst:
                lst.append(item)
    return lst
def get_movie_data(strng):
    url = 'http://www.omdbapi.com/'
    param = {'t':strng,'r':'json'}
    response = rq.get(url,param)
    final_response = response.json()
    return final_response

def get_movie_rating(dic):
    rating=0
    for items in dic['Ratings']:
        if items['Source']=='Rotten Tomatoes':
            rating = int(items['Value'].strip('%'))
    return rating

def get_sorted_recommendations(list_of_movies):
    dicto={}
    listo = get_related_titles(list_of_movies)
    for movies in listo:
        rating = get_movie_rating(get_movie_data(movies))
        dicto[movies]=rating
    return [movies[0] for movies in sorted(dicto.items(),key = lambda item: (item[1],item[0]),reverse=True)]
# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])

